In [1]:
import pandas as pd

import wandb

In [2]:
api = wandb.Api()

wandb: Currently logged in as: kwatcharasupat (kwatcharasupat-gatech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
import json
from pathlib import Path

results = []

for run in api.runs("kwatcharasupat-gatech/banda"):
    if run.group != "test runs - completed":
        continue

    print(run.name, run.state)
    config = run.config

    ckpt = config['ckpt_path']
    test_model_id = ckpt.split("/")[-3]

    if "test" not in config['data']:
        print("  No test set, skipping...")
        continue
    
    try:
        test_set = config['data']['test']['datasource'][0]['cls']
        print(test_model_id, test_set)
    except Exception as e:
        print(f"  Error getting test set: {e}, skipping...")
        continue

    save_path = Path(f"../results/{test_model_id}/{test_set}/detailed_results.csv")
    if save_path.exists():
        results.append(str(save_path))
        print(f"  {save_path} exists, skipping...")
        continue
    
    metrics = run.summary['test/metrics']
    table = run.logged_artifacts()[0]
    table_dir = table.download("../_artifacts")
    table_name = "test/metrics"
    table_path = f"{table_dir}/{table_name}.table.json"

    with open(table_path) as file:
        json_dict = json.load(file)

    df = pd.DataFrame(json_dict["data"], columns=json_dict["columns"])
    print(df.head())

    df['model'] = test_model_id
    df['test_set'] = test_set

    save_path.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(save_path, index=False)

    results.append(str(save_path))

bandit-mus64-l1snr-multi-adam-musdb18hq-vdbo-test-bass-1359 finished
lhtpigg8 MUSDB18HQDatasource
  ../results/lhtpigg8/MUSDB18HQDatasource/detailed_results.csv exists, skipping...
bandit-mus64-l1snr-multi-adam-moisesdb-vdbo-test-bass-1359 finished
lhtpigg8 MoisesDBDatasource
  ../results/lhtpigg8/MoisesDBDatasource/detailed_results.csv exists, skipping...
bandit-mus64-l1snr-multi-adam-musdb18hq-vdbo-test-other-1359 finished
2tx9mc4f MUSDB18HQDatasource
  ../results/2tx9mc4f/MUSDB18HQDatasource/detailed_results.csv exists, skipping...
bandit-mus64-l1snr-multi-adam-moisesdb-vdbo-test-drums-1359 finished
9rg6syhm MoisesDBDatasource
  ../results/9rg6syhm/MoisesDBDatasource/detailed_results.csv exists, skipping...
bandit-mus64-l1snr-multi-adam-musdb18hq-vdbo-test-vdbo-1359 finished
xzslzg26 MUSDB18HQDatasource
  ../results/xzslzg26/MUSDB18HQDatasource/detailed_results.csv exists, skipping...
bandit-mus64-l1snr-multi-adam-musdb18hq-vdbo-test-other-1359 finished
qtyn4z9v MUSDB18HQDatasource


wandb:   1 of 1 files downloaded.  


                                           full_path  vocals/PredDecibel  \
0  /home/hice1/kwatchar3/Documents/data/moisesdb/...          -33.221138   
1  /home/hice1/kwatchar3/Documents/data/moisesdb/...          -32.826504   
2  /home/hice1/kwatchar3/Documents/data/moisesdb/...          -27.719458   
3  /home/hice1/kwatchar3/Documents/data/moisesdb/...          -36.232872   
4  /home/hice1/kwatchar3/Documents/data/moisesdb/...          -25.147333   

   vocals/SNR  vocals/SignalNoiseRatio  vocals/TargetDecibel  \
0    5.543289                 5.543289            -33.488361   
1    8.124781                 8.124781            -32.111732   
2   14.171737                14.171737            -27.513449   
3    8.493666                 8.493666            -35.463284   
4    9.049488                 9.049488            -24.466867   

   drums/PredDecibel  drums/SNR  drums/SignalNoiseRatio  drums/TargetDecibel  \
0         -30.068552   4.211444                4.211444           -30.736063  

wandb:   1 of 1 files downloaded.  


                                           full_path  vocals/PredDecibel  \
0  /home/hice1/kwatchar3/Documents/data/moisesdb/...          -33.420254   
1  /home/hice1/kwatchar3/Documents/data/moisesdb/...          -32.814079   
2  /home/hice1/kwatchar3/Documents/data/moisesdb/...          -27.716751   
3  /home/hice1/kwatchar3/Documents/data/moisesdb/...          -36.208275   
4  /home/hice1/kwatchar3/Documents/data/moisesdb/...          -25.141699   

   vocals/SNR  vocals/SignalNoiseRatio  vocals/TargetDecibel  \
0    6.253423                 6.253423            -33.488361   
1    8.213115                 8.213115            -32.111732   
2   14.093578                14.093578            -27.513449   
3    8.312506                 8.312506            -35.463284   
4    8.998268                 8.998268            -24.466867   

   drums/PredDecibel  drums/SNR  drums/SignalNoiseRatio  drums/TargetDecibel  \
0         -30.322433   4.603183                4.603183           -30.736063  

In [4]:
df = pd.concat([pd.read_csv(f) for f in results], ignore_index=True)

timestamp = pd.Timestamp.now().strftime("%Y%m%d_%H%M%S")
# df.to_csv(f"../results/combined_detailed_results_{timestamp}.csv", index=False)

df = pd.melt(df, id_vars=['model', 'test_set', 'full_path'], var_name='metric', value_name='value')
df['test_stem'] = df['metric'].apply(lambda x: x.split('/')[0])
df['metric_name'] = df['metric'].apply(lambda x: x.split('/')[-1])


In [5]:
unique_models = df['model'].unique()

model_df = []


for test_model_id in unique_models:
    run = api.run(f"kwatcharasupat-gatech/banda/{test_model_id}")
    config = run.config

    epoch = run.summary['epoch']

    # print(run.name)

    model_cls = config['model']['cls']
    tfmodel_config = config['model']['params']['tf_model']
    tfmodel_config_dict = eval(tfmodel_config)
    tfmodel_cls = tfmodel_config_dict['cls']

    # print(" Model class:", model_cls)
    # print(" TF Model class:", tfmodel_cls) 

    loss_use_dbm = "dbm" in run.name
    loss_use_zlimit = "l1snrz" in run.name

    loss = "l1snr"
    if loss_use_zlimit:
        loss += "z"
    if loss_use_dbm:
        loss += "+dbm"

    stems = config['model']['params']['stems']
    if len(stems) == 1:
        stems = stems[0]
        model_cls = "Single" + model_cls
    else:
        stems = "".join([s[0] for s in stems])

    training_set = config['data']['train']['datasource'][0]['cls']
    # training_set = "MoisesDB" if "MoisesDB" in training_set else training_set


    pretrained_encoder  = config['model']['params'].get('pretrained_encoder_ckpt_path', None) is not None

    # print(" Stems:", stems)

    model_df.append({
        'model': test_model_id,
        'model_cls': model_cls,
        'tfmodel_cls': tfmodel_cls,
        'training_set': training_set,
        'epoch': epoch,
        'training_stems': stems,
        'pretrained_encoder': pretrained_encoder,
        'loss': loss
    })

model_df = pd.DataFrame(model_df)

In [6]:
id_cols = ['model', 'model_cls', 'tfmodel_cls', 'training_set', 'training_stems', 'pretrained_encoder', 'loss', 'epoch']
sort_cols = ['model_cls', 'tfmodel_cls', 'training_set', 'training_stems', 'pretrained_encoder', 'loss', 'epoch']

In [7]:
model_df = model_df.sort_values(sort_cols)[id_cols]

drop_ids = ['8akgy5kl', '6932jblk']

model_df = model_df[~model_df['model'].isin(drop_ids)]

model_df

,model,model_cls,tfmodel_cls,training_set,training_stems,pretrained_encoder,loss,epoch
15,prl820re,FixedStemBandit,MambaTFModel,MUSDB18HQDatasource,vdbo,False,l1snr,99
17,23wth3uc,FixedStemBandit,MambaTFModel,MoisesDBDatasource,vdbo,False,l1snr,99
12,zune532k,FixedStemBandit,RNNSeqBandModellingModule,MUSDB18HQDatasource,vdbo,False,l1snr,99
18,9qspypvq,FixedStemBandit,RNNSeqBandModellingModule,MUSDB18HQDatasource,vdbo,False,l1snr,249
3,xzslzg26,FixedStemBandit,RNNSeqBandModellingModule,MoisesDBDatasource,vdbo,False,l1snr,99
19,3zkxq8iu,FixedStemBandit,RNNSeqBandModellingModule,MoisesDBDatasource,vdbo,False,l1snr,249
21,uhnr1app,FixedStemBandit,RNNSeqBandModellingModule,MoisesDBStemWiseDatasource,vdbgpwbpooo,False,l1snr,99
22,gpxb68eo,FixedStemBandit,RNNSeqBandModellingModule,MoisesDBStemWiseDatasource,vdbgpwbpooo,False,l1snr+dbm,99
23,h5fcofk0,FixedStemBandit,RNNSeqBandModellingModule,MoisesDBStemWiseDatasource,vdbgpwbpooo,False,l1snrz+dbm,99
29,1neek7gq,FixedStemBandit,RNNSeqBandModellingModule,MoisesDBStemWiseDatasource,vdbgpwbpooo,False,l1snrz+dbm,99


In [8]:
# model_df = model_df[model_df.training_set.str.contains("MoisesDB")]  

# model_df

In [9]:
df = df.merge(model_df, on='model', how='inner')

In [10]:
df.columns

Index(['model', 'test_set', 'full_path', 'metric', 'value', 'test_stem',
       'metric_name', 'model_cls', 'tfmodel_cls', 'training_set',
       'training_stems', 'pretrained_encoder', 'loss', 'epoch'],
      dtype='object')

In [11]:
dfg = df.groupby(id_cols
        + ['test_set', 'test_stem', 'metric_name']
).median(numeric_only=True).reset_index()

dfg = dfg[dfg.metric_name == "SignalNoiseRatio"].dropna(subset=['value'])

dfg

,model,model_cls,tfmodel_cls,training_set,training_stems,pretrained_encoder,loss,epoch,test_set,test_stem,metric_name,value
2,0l0r6gav,FixedStemBandit,RoFormerTFModel,MUSDB18HQDatasource,vdbo,False,l1snr,99,MUSDB18HQDatasource,bass,SignalNoiseRatio,5.516438
10,0l0r6gav,FixedStemBandit,RoFormerTFModel,MUSDB18HQDatasource,vdbo,False,l1snr,99,MUSDB18HQDatasource,drums,SignalNoiseRatio,7.731935
18,0l0r6gav,FixedStemBandit,RoFormerTFModel,MUSDB18HQDatasource,vdbo,False,l1snr,99,MUSDB18HQDatasource,other,SignalNoiseRatio,4.970463
38,0l0r6gav,FixedStemBandit,RoFormerTFModel,MUSDB18HQDatasource,vdbo,False,l1snr,99,MUSDB18HQDatasource,vocals,SignalNoiseRatio,7.193657
46,0l0r6gav,FixedStemBandit,RoFormerTFModel,MUSDB18HQDatasource,vdbo,False,l1snr,99,MoisesDBDatasource,bass,SignalNoiseRatio,8.854338
...,...,...,...,...,...,...,...,...,...,...,...,...
2106,zune532k,FixedStemBandit,RNNSeqBandModellingModule,MUSDB18HQDatasource,vdbo,False,l1snr,99,MUSDB18HQDatasource,vocals,SignalNoiseRatio,7.866929
2114,zune532k,FixedStemBandit,RNNSeqBandModellingModule,MUSDB18HQDatasource,vdbo,False,l1snr,99,MoisesDBDatasource,bass,SignalNoiseRatio,8.062817
2122,zune532k,FixedStemBandit,RNNSeqBandModellingModule,MUSDB18HQDatasource,vdbo,False,l1snr,99,MoisesDBDatasource,drums,SignalNoiseRatio,8.073868
2130,zune532k,FixedStemBandit,RNNSeqBandModellingModule,MUSDB18HQDatasource,vdbo,False,l1snr,99,MoisesDBDatasource,other,SignalNoiseRatio,5.141108


In [12]:
dfg.test_stem.unique()

array(['bass', 'drums', 'other', 'vocals', 'bowed_strings', 'guitar',
       'other_keys', 'other_plucked', 'percussion', 'piano', 'wind'],
      dtype=object)

In [13]:
stem_order = [
    'vocals', 'drums', 'bass', 'guitar', 'piano', 'wind', 'bowed_strings', 'percussion', 'other_keys', 'other_plucked', 'other'
]
dfg['test_stem'] = pd.Categorical(dfg['test_stem'], categories=stem_order, ordered=True)

dfg = dfg.dropna()

In [14]:
dfg.columns

Index(['model', 'model_cls', 'tfmodel_cls', 'training_set', 'training_stems',
       'pretrained_encoder', 'loss', 'epoch', 'test_set', 'test_stem',
       'metric_name', 'value'],
      dtype='object')

In [34]:
dfg

,model,model_cls,tfmodel_cls,training_set,training_stems,pretrained_encoder,loss,epoch,test_set,test_stem,metric_name,value
2,0l0r6gav,FixedStemBandit,RoFormerTFModel,MUSDB18HQDatasource,vdbo,False,l1snr,99,MUSDB18HQDatasource,bass,SignalNoiseRatio,5.516438
10,0l0r6gav,FixedStemBandit,RoFormerTFModel,MUSDB18HQDatasource,vdbo,False,l1snr,99,MUSDB18HQDatasource,drums,SignalNoiseRatio,7.731935
18,0l0r6gav,FixedStemBandit,RoFormerTFModel,MUSDB18HQDatasource,vdbo,False,l1snr,99,MUSDB18HQDatasource,other,SignalNoiseRatio,4.970463
38,0l0r6gav,FixedStemBandit,RoFormerTFModel,MUSDB18HQDatasource,vdbo,False,l1snr,99,MUSDB18HQDatasource,vocals,SignalNoiseRatio,7.193657
46,0l0r6gav,FixedStemBandit,RoFormerTFModel,MUSDB18HQDatasource,vdbo,False,l1snr,99,MoisesDBDatasource,bass,SignalNoiseRatio,8.854338
...,...,...,...,...,...,...,...,...,...,...,...,...
2106,zune532k,FixedStemBandit,RNNSeqBandModellingModule,MUSDB18HQDatasource,vdbo,False,l1snr,99,MUSDB18HQDatasource,vocals,SignalNoiseRatio,7.866929
2114,zune532k,FixedStemBandit,RNNSeqBandModellingModule,MUSDB18HQDatasource,vdbo,False,l1snr,99,MoisesDBDatasource,bass,SignalNoiseRatio,8.062817
2122,zune532k,FixedStemBandit,RNNSeqBandModellingModule,MUSDB18HQDatasource,vdbo,False,l1snr,99,MoisesDBDatasource,drums,SignalNoiseRatio,8.073868
2130,zune532k,FixedStemBandit,RNNSeqBandModellingModule,MUSDB18HQDatasource,vdbo,False,l1snr,99,MoisesDBDatasource,other,SignalNoiseRatio,5.141108


In [35]:
# dfg_vdbo = dfg[dfg.test_stem.isin(['vocals', 'drums', 'bass', 'other'])]
# dfg_vdbo = dfg[dfg.test_set == "MoisesDBDatasource"]
# dfg_vdbo = dfg[dfg.test_set == "MUSDB18HQDatasource"]
dfg_vdbo = dfg[dfg.epoch == 149]

In [37]:
dfgx = pd.pivot(
    dfg_vdbo,
    index=['model_cls', 'tfmodel_cls', 'training_stems', 'pretrained_encoder', 'epoch', 'loss', 'training_set', 'model'],
    columns=['test_set', 'test_stem'],
    values='value'
).sort_index(
    axis=1,
    level=[0,1],
    ascending=[True, True]
).reset_index()

# print(dfgx.round(2).to_latex(index=False, float_format="%.2f"))

In [36]:
dfgx.to_csv(f"../results/summary_results_{timestamp}.csv", index=False)

In [38]:
dfgx.round(1)


test_set               model_cls                tfmodel_cls training_stems  \
test_stem                                                                    
0          VectorDictQueryBandit  RNNSeqBandModellingModule           vdbo   
1          VectorDictQueryBandit  RNNSeqBandModellingModule           vdbo   

test_set  pretrained_encoder epoch        loss         training_set     model  \
test_stem                                                                       
0                      False   149  l1snrz+dbm  MUSDB18HQDatasource  acgtdtye   
1                      False   149  l1snrz+dbm  MUSDB18HQDatasource  dh3ryeki   

test_set  MoisesDBDatasource                    
test_stem             vocals drums  bass other  
0                        8.7  10.0  10.2   6.4  
1                        8.7  10.0  10.2   6.3